#### Code comes from the official readme file

In [8]:
import pandas as pd

df = pd.read_csv('../archive/Edge-IIoTset dataset/Selected dataset for Ml and DL/DNN-EdgeIIoT-dataset.csv')

C:\Users\elyes\AppData\Local\Temp\ipykernel_18508\1311819357.py:3: DtypeWarning: Columns (2,3,6,11,13,14,15,16,17,31,32,34,39,45,51,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../archive/Edge-IIoTset dataset/Selected dataset for Ml and DL/DNN-EdgeIIoT-dataset.csv')


In [9]:
from sklearn.utils import shuffle

drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 
         "http.file_data","http.request.full_uri","icmp.transmit_timestamp",
         "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",
         "tcp.dstport", "udp.port", "mqtt.msg","icmp.unused","dns.qry.type","http.tls_port","mqtt.msg_decoded_as"]

df.drop(drop_columns, axis=1, inplace=True)

df.dropna(axis=0, how='any', inplace=True)

df.drop_duplicates(subset=None, keep="first", inplace=True)

df = shuffle(df)

df.isna().sum().sum()

0

In [10]:
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])

    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]

    df.drop(name, axis=1, inplace=True)

encode_text_dummy(df,'http.request.method')
encode_text_dummy(df,'http.referer')
encode_text_dummy(df,"http.request.version")
encode_text_dummy(df,"dns.qry.name.len")
encode_text_dummy(df,"mqtt.conack.flags")
encode_text_dummy(df,"mqtt.protoname")
encode_text_dummy(df,"mqtt.topic")

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Attack_type'] = le.fit_transform(df['Attack_type'])
le.classes_

array(['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP',
       'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning',
       'Ransomware', 'SQL_injection', 'Uploading',
       'Vulnerability_scanner', 'XSS'], dtype=object)

In [12]:
for feature in df.columns:
    if df[feature].nunique() == 1:
        print(f"{feature}: {df[feature].nunique()}")

In [13]:
from sklearn.preprocessing import StandardScaler

for feature in df.columns.drop(['Attack_type','Attack_label']):
    df[feature] = StandardScaler().fit_transform(df[feature].values.reshape(-1, 1))
df.columns = df.columns.str.replace('[\[\],<]', '', regex=True)

In [18]:
df['Attack_type'].nunique()

15

In [7]:
df.to_parquet('../archive/preprocessed_DNN.parquet')